In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [10]:
true_df = pd.read_csv('/Users/shauryabhagat/Desktop/GUTS/True.csv', header=0)
fake_df = pd.read_csv('/Users/shauryabhagat/Desktop/GUTS/Fake.csv', header=0)

In [11]:
true_df['label'] = 0
fake_df['label'] = 1

In [12]:
combined_df = pd.concat([fake_df,true_df])

In [13]:
combined_df = combined_df.sample(frac=1).reset_index(drop=True)

In [14]:
combined_df = combined_df.fillna(' ')

In [15]:
stoplist = set(stopwords.words('english'))

In [16]:
combined_df['text'] = combined_df['text'].apply(word_tokenize)
combined_df['text'] = combined_df['text'].apply(lambda x: [each_word for each_word in x if each_word not in stoplist])
combined_df['text_joined'] = combined_df['text'].apply(lambda x: " ".join(x))

In [64]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
tf_idf_transformer = TfidfTransformer(smooth_idf=False)

In [65]:
count_vectorizer.fit(combined_df['text_joined'].values)
count_vect_train = count_vectorizer.transform(combined_df['text_joined'].values)

In [66]:
tf_idf_transformer.fit(count_vect_train)
tf_idf_train = tf_idf_transformer.transform(count_vect_train)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_train, combined_df['label'], random_state=0)

In [23]:
lr = LogisticRegression()

In [24]:
lr.fit(X_train, y_train)

LogisticRegression()

In [25]:
pred = lr.predict(X_test)

In [26]:
print("Accuracy_Training_Set = " , lr.score(X_train, y_train)*100)
print("Accuracy_Test_Set = " , lr.score(X_test, y_test)*100)
print("Precision = " , precision_score(y_test, pred)*100)
print("Recall = " , recall_score(y_test, pred)*100)
print("F1 score = " , f1_score(y_test, pred)*100)

Accuracy_Training_Set =  99.40605232679
Accuracy_Test_Set =  98.52115812917594
Precision =  98.42905405405405
Recall =  98.76271186440678
F1 score =  98.59560067681896


In [55]:
kg_df = pd.read_csv('/Users/shauryabhagat/Desktop/GUTS/KG_train.csv', header=0)

In [56]:
kg_df = kg_df.replace(0,99)  #0 = barely true, which will be removed
kg_df = kg_df.replace(5,0)   # because 5 is true

In [57]:
indexNames = kg_df[ (kg_df['Labels'] >= 2)] .index # or (kg_df['Labels'] == 3) or (kg_df['Labels'] == 4) or (kg_df['Labels'] == 99) ].index
kg_df.drop(indexNames , inplace=True)

In [58]:
kg_df = kg_df.fillna(' ')

In [59]:
kg_df['Text'] = kg_df['Text'].apply(word_tokenize)
kg_df['Text'] = kg_df['Text'].apply(lambda x: [each_word for each_word in x if each_word not in stoplist])
kg_df['text_joined'] = kg_df['Text'].apply(lambda x: " ".join(x))

In [67]:
count_vect_kg = count_vectorizer.transform(kg_df['text_joined'].values)
tf_idf_kg = tf_idf_transformer.transform(count_vect_kg)

In [69]:
kg_pred = lr.predict(tf_idf_kg)

In [71]:
print("Accuracy = ", accuracy_score(kg_df['Labels'], kg_pred)*100)
print("Precision = " , precision_score(kg_df['Labels'], kg_pred)*100)
print("Recall = " , recall_score(kg_df['Labels'], kg_pred)*100)
print("F1 score = " , f1_score(kg_df['Labels'], kg_pred)*100)

accuracy =  54.12694088804141
Precision =  54.35696273465956
Recall =  97.24310776942356
F1 score =  69.73400431344358
